In [1]:
import Crypto
from Crypto.Cipher import AES

In [2]:
import sys

In [3]:
def embed(sentence, wm, password):
    # 对水印做AES加密
    wm = wm.encode('utf-8')
    cryptor = AES.new(key='{:0<16}'.format(password).encode('utf-8'),
                      mode=AES.MODE_ECB)  # key 长度必须是16,24,32 长度的 byte 格式
    ciphertext_tmp = cryptor.encrypt(wm + b' ' * (16 - len(wm) % 16))  # 明文的长度必须是16的整数倍
    ciphertext_hex = ciphertext_tmp.hex()  # 转为16进制

    bin_text = bin(int(ciphertext_hex, base=16))[2:]

    # 打入水印
    len_bin_text = len(bin_text)
    len_sentence = len(sentence)
    assert len_sentence > len_bin_text, "文本长度太短了,至少{}".format(len_bin_text)

    sentence_embed = ""
    for idx in range(len_sentence):
        sentence_embed += sentence[idx]
        if idx < len_bin_text:
            if bin_text[idx] == "1":
                sentence_embed += chr(127)

    # print("打入水印后的结果：")
    # print("水印长度{}".format(len(bin_text)))
    # print(sentence_embed)
    return sentence_embed

In [5]:
wm = "绝密：两点老地方见！"
password = '20190808'
sentence = "这句话中有盲水印，你能提取出来吗？" * 16

In [6]:
# %%
# 打入盲水印
sentence_embed = embed(sentence, wm, password)

In [7]:
sentence_embed

'这\x7f句话\x7f中\x7f有\x7f盲\x7f水\x7f印\x7f，你能提取\x7f出来\x7f吗\x7f？这句话\x7f中有盲水\x7f印，\x7f你能提\x7f取出\x7f来吗？这\x7f句话中\x7f有盲\x7f水印，你\x7f能\x7f提取\x7f出\x7f来\x7f吗？这句话中有盲水\x7f印，\x7f你能\x7f提取\x7f出\x7f来吗？\x7f这\x7f句话\x7f中有\x7f盲水印，\x7f你\x7f能提取出来\x7f吗？\x7f这句话\x7f中\x7f有\x7f盲水\x7f印\x7f，\x7f你能提取\x7f出来吗？\x7f这\x7f句\x7f话\x7f中有\x7f盲水印\x7f，\x7f你能提取出来吗？这句话中有盲\x7f水印，\x7f你\x7f能\x7f提\x7f取出\x7f来吗？\x7f这句\x7f话中有盲水\x7f印，你能\x7f提\x7f取出\x7f来吗\x7f？这\x7f句话\x7f中\x7f有盲水印，你\x7f能\x7f提取\x7f出来\x7f吗\x7f？这句话\x7f中\x7f有盲\x7f水印，\x7f你\x7f能\x7f提取\x7f出来\x7f吗？这\x7f句话\x7f中有\x7f盲\x7f水印\x7f，\x7f你\x7f能\x7f提取出来\x7f吗？这\x7f句\x7f话\x7f中有盲\x7f水\x7f印，你能提取\x7f出来\x7f吗？这\x7f句话中\x7f有\x7f盲水印\x7f，你能\x7f提取\x7f出\x7f来\x7f吗\x7f？这句\x7f话中\x7f有\x7f盲\x7f水印\x7f，你\x7f能提\x7f取出来\x7f吗\x7f？这句话中有盲水印，你能提取出来吗？'

In [4]:
# %%提取水印

def extract(sentence_embed, password):
    bin_wm_extract = ""
    previous_is_char = False
    for i in sentence_embed:
        if previous_is_char:
            if ord(i) == 127:
                bin_wm_extract += "1"
                previous_is_char = False
            else:
                bin_wm_extract += "0"
                previous_is_char = True
        else:
            previous_is_char = True

    #  去掉末尾的0

    last_zero = len(bin_wm_extract) - bin_wm_extract[::-1].find("1")

    last_zero = ((last_zero - 1) // 128 + 1) * 128
    bin_wm_extract = bin_wm_extract[:last_zero]

    # 解密

    hex_wm_extract = hex(int(bin_wm_extract, base=2))

    return AES.new(key='{:0<16}'.format(password).encode('utf-8'), mode=AES.MODE_ECB) \
        .decrypt(bytes.fromhex(hex_wm_extract[2:])).decode('utf-8')  # key 长度必须是16,24,32 长度的 byte 格式

In [8]:
# %% 解出盲水印
wm_extract = extract(sentence_embed, password)

In [9]:
wm_extract

'绝密：两点老地方见！  '